In [2]:
# 自定义层
# 不带参数的层
''' 
⾸先，我们构造⼀个没有任何参数的⾃定义层。如果你还记得我们在 5.1节对块的介绍，这应该看起来很眼
熟。下⾯的CenteredLayer类要从其输⼊中减去均值。要构建它，我们只需继承基础层类并实现前向传播
功能
'''
import torch
import torch.nn.functional as F
from torch import nn
class CenteredLayer(nn.Module): # 创建层类 只需继承基础层并实现前向传播
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean() # 从输入中减去均值

In [3]:
# 让我们向该层提供⼀些数据，验证它是否能按预期⼯作
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5])) # 各值减去均值

tensor([-2., -1.,  0.,  1.,  2.])

In [4]:
# 现在，我们可以将层作为组件合并到更复杂的模型中。
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [5]:
''' 
作为额外的健全性检查，我们可以在向该⽹络发送随机数据后，检查均值是否为0。由于我们处理的是浮点
数，因为存储精度的原因，我们仍然可能会看到⼀个⾮常⼩的⾮零数
'''
Y = net(torch.rand(4, 8)) # rand 返回一个由区间上[0, 1) 均匀分布的随机数填充的张量。 size 4行8列
Y.mean()

tensor(-4.6566e-09, grad_fn=<MeanBackward0>)

In [8]:
# 带参数的层、
''' 
以上我们知道了如何定义简单的层，下⾯我们继续定义具有参数的层，这些参数可以通过训练进⾏调整。我
们可以使⽤内置函数来创建参数，这些函数提供⼀些基本的管理功能。⽐如管理访问、初始化、共享、保存
和加载模型参数。这样做的好处之⼀是：我们不需要为每个⾃定义层编写⾃定义的序列化程序。
现在，让我们实现⾃定义版本的全连接层。回想⼀下，该层需要两个参数，⼀个⽤于表⽰权重，另⼀个⽤于
表⽰偏置项。在此实现中，我们使⽤修正线性单元作为激活函数。该层需要输⼊参数：in_units和units，
分别表⽰输⼊数和输出数
'''
class MyLinear(nn.Module):
    def __init__(self, in_units, units): # in_units和units，分别表⽰输⼊数和输出数
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) # 正态分布
        self.bias = nn.Parameter(torch.randn(units,)) 
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data # 全连接层
        return F.relu(linear)

In [7]:
# 接下来，我们实例化MyLinear类并访问其模型参数。
linear = MyLinear(5, 3) 
linear.weight

Parameter containing:
tensor([[-0.5371,  1.4096,  0.7424],
        [ 0.3159,  0.8673,  0.3617],
        [ 1.2633, -0.0129, -0.2248],
        [-0.2889,  0.0317,  1.9702],
        [-0.3716, -0.4693, -0.2003]], requires_grad=True)

In [9]:
# 我们可以使⽤⾃定义层直接执⾏前向传播计算。
linear(torch.rand(2, 5)) # 均匀分布 输入 size = (2,5) 

tensor([[0.0000, 1.9266, 0.6615],
        [0.0000, 2.1338, 1.9508]])

In [10]:
# 我们还可以使⽤⾃定义层构建模型，就像使⽤内置的全连接层⼀样使⽤⾃定义层。
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [0.1508]])

In [ ]:
''' 
⼩结
• 我们可以通过基本层类设计⾃定义层。这允许我们定义灵活的新层，其⾏为与深度学习框架中的任何现有层不同。
• 在⾃定义层定义完成后，我们就可以在任意环境和⽹络架构中调⽤该⾃定义层。
• 层可以有局部参数，这些参数可以通过内置函数创建。
'''